## 1. Tools for text processing
<p><img style="float: right ; margin: 5px 20px 5px 10px; width: 45%" src="https://assets.datacamp.com/production/project_38/img/Moby_Dick_p510_illustration.jpg"> </p>
<p>What are the most frequent words in Herman Melville's novel, Moby Dick, and how often do they occur?</p>
<p>In this notebook, we'll scrape the novel <em>Moby Dick</em> from the website <a href="https://www.gutenberg.org/">Project Gutenberg</a> (which contains a large corpus of books) using the Python package <code>requests</code>. Then we'll extract words from this web data using <code>BeautifulSoup</code>. Finally, we'll dive into analyzing the distribution of words using the Natural Language ToolKit (<code>nltk</code>) and <code>Counter</code>.</p>
<p>The <em>Data Science pipeline</em> we'll build in this notebook can be used to visualize the word frequency distributions of any novel that you can find on Project Gutenberg. The natural language processing tools used here apply to much of the data that data scientists encounter as a vast proportion of the world's data is unstructured data and includes a great deal of text.</p>
<p>Let's start by loading in the three main Python packages we are going to use.</p>

In [7]:
# Importing requests, BeautifulSoup, nltk, and Counter
import requests
from bs4 import BeautifulSoup
import nltk
from collections import Counter
import pprint

## 2. Request Moby Dick
<p>To analyze Moby Dick, we need to get the contents of Moby Dick from <em>somewhere</em>. Luckily, the text is freely available online at Project Gutenberg as an HTML file: https://www.gutenberg.org/files/2701/2701-h/2701-h.htm .</p>
<p><strong>Note</strong> that HTML stands for Hypertext Markup Language and is the standard markup language for the web.</p>
<p>To fetch the HTML file with Moby Dick we're going to use the <code>request</code> package to make a <code>GET</code> request for the website, which means we're <em>getting</em> data from it. This is what you're doing through a browser when visiting a webpage, but now we're getting the requested page directly into Python instead. </p>

In [13]:
#Getting the Moby dick HTML
request = requests.get('https://s3.amazonaws.com/assets.datacamp.com/production/project_147/datasets/2701-h.htm')

#Setting the correct text encoding of HTML page
request.encoding = "utf-8"

#Extracting the HTML from request object
html = request.text

#Printing the first 500 characters in HTML
print(html[:500])

<?xml version="1.0" encoding="utf-8"?>

<!DOCTYPE html
   PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"
   "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd" >

<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
  <head>
    <title>
      Moby Dick; Or the Whale, by Herman Melville
    </title>
    <style type="text/css" xml:space="preserve">

    body { background:#faebd0; color:black; margin-left:15%; margin-right:15%; text-align:justify }
    P { text-indent: 1em; margin-top: .2


## 3. Get the text from the HTML
<p>This HTML is not quite what we want. However, it does <em>contain</em> what we want: the text of <em>Moby Dick</em>. What we need to do now is <em>wrangle</em> this HTML to extract the text of the novel. For this we'll use the package <code>BeautifulSoup</code>.</p>
<p>Firstly, a word on the name of the package: Beautiful Soup? In web development, the term "tag soup" refers to structurally or syntactically incorrect HTML code written for a web page. What Beautiful Soup does best is to make tag soup beautiful again and to extract information from it with ease! In fact, the main object created and queried when using this package is called <code>BeautifulSoup</code>.</p>

In [16]:
#Creating a BeautifulSoup object from HTML
soup = BeautifulSoup(html,"html.parser")

#Getting the text out of the soup
text = soup.get_text()

print(text[100:1000])

Moby Dick; or The Whale, by Herman Melville

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Moby Dick; or The Whale

Author: Herman Melville

Release Date: December 25, 2008 [EBook #2701]
Last Updated: December 3, 2017

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK MOBY DICK; OR THE WHALE ***




Produced by Daniel Lazarus, Jonesey, and David Widger









      MOBY-DICK;or, THE WHALE.
    




      By Herman Melville
    

 



 



CONTENTS





 ETYMOLOGY. 


 EXTRACTS (Supplied by a Sub-Sub-Librarian).
        





 CHAPTER 1. Loomings. 


 CHAPTER 2. The Carpet-Bag. 


 CHAPTER 3. The Spouter


## 4. Extract the words
<p>We now have the text of the novel! There is some unwanted stuff at the start and some unwanted stuff at the end. We could remove it, but this content is so much smaller in amount than the text of Moby Dick that, to a first approximation, it is okay to leave it in.</p>
<p>Now that we have the text of interest, it's time to count how many times each word appears, and for this we'll use <code>nltk</code> – the Natural Language Toolkit. We'll start by tokenizing the text, that is, remove everything that isn't a word (whitespace, punctuation, etc.) and then split the text into a list of words.</p>

In [30]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
tokens = tokenizer.tokenize(text)

pprint.pprint(tokens[:8])

['Moby', 'Dick', ';', 'Or', 'the', 'Whale', ',', 'by']


## 5. Make the words lowercase
<p>OK! We're nearly there. Note that in the above 'Or' has a capital 'O' and that in other places it may not, but both 'Or' and 'or' should be counted as the same word. For this reason, we should build a list of all words in <em>Moby Dick</em> in which all capital letters have been made lower case.</p>

In [31]:
import string
#Update tokens  list with lower-casing and removing punctuation
tokens = [token.lower() for token in tokens if token not in string.punctuation]
#print out first 8 words /tokens
pprint.pprint(tokens[:8])

['moby', 'dick', 'or', 'the', 'whale', 'by', 'herman', 'melville']


## 6. Load in stop words
<p>It is common practice to remove words that appear a lot in the English language such as 'the', 'of' and 'a' because they're not so interesting. Such words are known as <em>stop words</em>. The package <code>nltk</code> includes a good list of stop words in English that we can use.</p>

In [43]:
# Getting the English stop words from nltk
stopwords = nltk.corpus.stopwords.words("english")
stopwords.append(".”")
stopwords.append("’s")

# Printing out the first eight stop words
stopwords[:8]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves']

## 7. Remove stop words in Moby Dick
<p>We now want to create a new list with all <code>words</code> in Moby Dick, except those that are stop words (that is, those words listed in <code>sw</code>).</p>

In [44]:
# Create a list words_ns containing all words that are in words but not in sw
words =[word for word in tokens if word not in stopwords]

#printing the first 5 words 
pprint.pprint(words[:8])

['moby', 'dick', 'whale', 'herman', 'melville', 'project', 'gutenberg', 'ebook']


## 8. We have the answer
<p>Our original question was:</p>
<blockquote>
  <p>What are the most frequent words in Herman Melville's novel Moby Dick and how often do they occur?</p>
</blockquote>
<p>We are now ready to answer that! Let's answer this question using the <code>Counter</code> class we imported earlier.</p>

In [45]:
#Initilize a Counter object from processed list of words
count = Counter(words)

#Store 10 most common words and their counts as top_ten
top_ten = count.most_common(10)

# Print the top ten words and their counts
pprint.pprint(top_ten)

[('whale', 1233),
 ('one', 914),
 ('like', 569),
 ('upon', 564),
 ('ahab', 505),
 ('man', 505),
 ('ship', 479),
 ('sea', 450),
 ('old', 449),
 ('ye', 431)]


## 9. The most common word
<p>Nice! Using our variable <code>top_ten</code>, we now have an answer to our original question.</p>
<p>The natural language processing skills we used in this notebook are also applicable to much of the data that Data Scientists encounter as the vast proportion of the world's data is unstructured data and includes a great deal of text. </p>
<p>So, what word turned out to (<em>not surprisingly</em>) be the most common word in Moby Dick?</p>

In [46]:
# What's the most common word in Moby Dick?
most_common_word = 'whale'